In [1]:
##DataSet + AutoTrader
import numpy as np

##DataSet
from sklearn import preprocessing
from sklearn.utils import shuffle
import json

##Model
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from sklearn.metrics import accuracy_score

##AutoTrader
import time


### Xây dựng Class

In [2]:
DATASET_DIR = "datasets/"
COIN_PAIR = "BTC-USD"

TRAINING_MONTHS = ["2018_06","2018_07","2018_08","2018_09","2018_10","2018_11","2018_12","2019_01",
                   "2019_02","2019_03","2019_04","2019_05","2019_06","2019_07","2019_08","2019_09",
                   "2019_10","2019_11","2019_12","2020_01","2020_02","2020_03","2020_04","2020_05",
                    "2020_06","2020_07"]

TESTING_MONTHS = ["2020_08","2020_09","2020_10"]

TRAINING_WINDOW = 360
LABELING_WINDOW = 360 

+ Khai báo địa chỉ thư mục chứa dữ liệu, tên đồng tiền <br>
+ Chia các tháng cần train và các tháng để test <br>


In [3]:
class Dataset:

    def __init__(self):
        print("> Dataset Class Initialized")
        self.feature_names = []

    def loadCoinData(self,coin_name,months):
        print("> Loading data for ",coin_name)
        ordered_data = []
        for month in months:
            print(">> Loading month: ",month)
            file_path = DATASET_DIR + coin_name + '/' + month +"__"+ coin_name + '.json'
            with open(file_path) as json_file:
                raw_data = json.load(json_file)

            for data in raw_data:
                if len(data) != 0:
                    for minute_info in data:
                        ordered_data.append(minute_info)

        return ordered_data

    def createTrainTestSets(self,coin_name, coin_data, training_window = 180, labeling_window = 60, feature_window = 30):
        '''
        Khởi tạo :
        x chứa dữ liệu input (các cột của bộ dữ liệu)
        y chứa dữ liệu output (chứa nhãn 0,1 tượng trưng lúc giảm hay tăng)
        prices chứa giá trị đóng cuối cùng trong 1 phút
        
        positive: tổng số lần giá tăng
        negative: tổng số lần giá giảm
        block: số phút tăng lên mỗi vòng lặp 
        start_index: index bắt đầu
        end_index: index kết thúc
        '''
        
        print("> Creating X,Y sets for ",coin_name)
        x = []
        y = []
        prices = []

        positive = 0
        negative = 0
        block = 60
        start_index = 0
        end_index = training_window -1

        while end_index < len(coin_data) - labeling_window -block - 1:
            '''
            features: chứa giá trị cột
            featrues_names: chứa tên cột
            '''
            features = []
            self.feature_names = []
            
            # Lấy giá trị mới nhất sau 60 phút
            latest_low_price = coin_data[end_index][1]
            latest_high_price = coin_data[end_index][2]
            latest_open_price = coin_data[end_index][3]
            latest_close_price = coin_data[end_index][4]
            latest_volume = coin_data[end_index][5]

            '''
            acc_index: index trong khoảng thời gian [start,end]
            total_window_volume: tổng số giá trị giao dịch trong khoảng [start,end]
            prev_window_volume_acceleration: tổng số giá trị giao dịch trong khoảng [start,end] của feature_window trước đó
            window_volume_acceleration: tổng số giá trị giao dịch trong khoảng [start,end] của feature_window hiện tại
            average_volume_acceleration: trung bình tỉ lệ giao dịch trong 1 training_window
            average_close_price_acceleration: trung bình giá đóng cửa trong 1 training_window
            average_close_price_diff: trung bình độ sai lệch giá đóng trong 1 training_window
            '''
            acc_index = 0
            total_window_volume = 0
            prev_window_volume_acceleration = 0
            window_volume_acceleration = 0
            average_volume_acceleration = 0
            average_close_price_acceleration = 0
            average_close_price_diff = 0

            average_close_price = 0

            # Tính toán cột
            for i in range(start_index,end_index):
                cur_close_price = coin_data[i][4]
                cur_volume = coin_data[i][5]
                total_window_volume += cur_volume
                window_volume_acceleration += cur_volume
                average_close_price += cur_close_price

                acc_index += 1
                if acc_index % feature_window == 0:
                    
                    close_price_acceleration = (cur_close_price / coin_data[i-feature_window][4]) - 1
                    features.append(close_price_acceleration)
                    self.feature_names.append("close_price_acc_"+str(acc_index))
                    average_close_price_acceleration += close_price_acceleration

                    close_price_diff = cur_close_price - coin_data[i-feature_window][4]
                    average_close_price_diff +=  close_price_diff

                    if prev_window_volume_acceleration != 0:
                        volume_acceleration = (window_volume_acceleration / prev_window_volume_acceleration) - 1
                        features.append(volume_acceleration)
                        self.feature_names.append("volume_acc_"+str(acc_index))
                    average_volume_acceleration += window_volume_acceleration
                    prev_window_volume_acceleration = window_volume_acceleration
                    window_volume_acceleration = 0

            average_close_price /= training_window
            average_close_price_acceleration /= training_window
            average_volume_acceleration /= training_window

            overall_price_difference = latest_close_price - coin_data[start_index][4]
            overall_price_acceleration = (latest_close_price / coin_data[start_index][4]) - 1

            #Thêm giá trị cột 
            features.append(total_window_volume)
            self.feature_names.append("volume_total")
            features.append(latest_low_price)
            self.feature_names.append("low_price_latest")
            features.append(latest_high_price)
            self.feature_names.append("high_price_latest")
            features.append(latest_open_price)
            self.feature_names.append("open_price_latest")

            features.append(average_close_price)
            self.feature_names.append("close_price_av")
            features.append(average_close_price_acceleration)
            self.feature_names.append("close_price_acc_av")
            features.append(average_close_price_diff)
            self.feature_names.append("close_price_diff_av")
            features.append(average_volume_acceleration)
            self.feature_names.append("volume_acc_av")
            features.append(overall_price_difference)
            self.feature_names.append("overall_price_diff")
            features.append(overall_price_acceleration)
            self.feature_names.append("overal_price_acc")

            features.append(latest_close_price)
            self.feature_names.append("close_price_latest")
            
            prices.append(latest_close_price)

            x.append(features)

            '''
            next_close_price: lấy giá trị đóng của labeling_window tiếp theo
            change_rate: tỉ lệ thay đổi giá sắp tới = giá trị đóng sắp tới/ giá trị đóng hiện tại -1
            trường hợp (giá trị đóng sắp tới > giá trị đóng hiện tại) => change_rate > 0
            trường hợp (giá trị đóng sắp tới < giá trị đóng hiện tại) => change_rate < 0
            từ change_rate ta có thể phân loại giá trị cho y
            nhãn 0: giá giảm change_rate < 0
            nhãn 1: giá tăng change_rate > 0
            '''
            next_close_price = coin_data[end_index + labeling_window +1][4]

            change_rate = (next_close_price/latest_close_price) - 1

            if change_rate > 0:
                y.append(1)
                positive += 1
            else:
                y.append(0)
                negative +=1

            #Dịch khoảng [start,end] 1 đoạn block (60)
            start_index += block
            end_index += block

        print("> Finished Creating set - Size: ",len(x)," ",len(y)," P: ",positive," N: ",negative)

        #Chuẩn hóa dữ liệu vào trộn dữ liệu
        x = preprocessing.scale(x)
        x, y, prices = shuffle(x,y,prices)

        return np.array(x), np.array(y), prices

Tạo class Dataset có vai trò: <br>
+ Load dữ liệu theo tháng
+ Tạo cột cho bộ dữ liệu: <br>
['close_price_acc_120',
 'close_price_acc_150',
 'close_price_acc_180',
 'close_price_acc_210',
 'close_price_acc_240',
 'close_price_acc_270',
 'close_price_acc_30',
 'close_price_acc_300',
 'close_price_acc_330',
 'close_price_acc_60',
 'close_price_acc_90',
 'close_price_acc_av',
 'close_price_av',
 'close_price_diff_av',
 'close_price_latest',
 'high_price_latest',
 'low_price_latest',
 'open_price_latest',
 'overal_price_acc',
 'overall_price_diff',
 'volume_acc_120',
 'volume_acc_150',
 'volume_acc_180',
 'volume_acc_210',
 'volume_acc_240',
 'volume_acc_270',
 'volume_acc_300',
 'volume_acc_330',
 'volume_acc_60',
 'volume_acc_90',
 'volume_acc_av',
 'volume_av',
 'volume_total']
+ Ý nghĩa sinh các cột chủ yếu xoay quanh ý tưởng tận dụng các điểm dữ liệu trước đó trong qua khứ và sự dụng độ lệch, moving average loại bỏ noise để góp phần dự đoán
+ Tạo thêm cột cần dự đoán với nhãn 0, 1 tương ứng lúc bán (giá giảm), lúc mua (giá tăng)

In [4]:
class Model:

    def __init__(self,model_name,x_train):
        self.model_name = model_name
        self.model = self.buildModel(x_train)
        print("> New model initialized: ",model_name)


    def buildModel(self,x_train):
        model = Sequential()
        x = len(x_train[0])

        '''
        Xây dựng cấu trúc network:
        Với 3 lớp LSTM số node giảm dần do LSTM xử lý tốt với các dữ liệu sequence/time-series nên tăng số lớp để tăng độ chính xác
        Sử dụng hàm relu để tốc độ dự đoán nhanh hơn 
        Bởi vì mô hình dùng để phân loại nên sử dùng hàm softmax ở layer output
        '''
        model.add(LSTM(256,input_shape=((1,x)), return_sequences = True, activation = "relu"))
        model.add(Dropout(0.2))
        model.add(BatchNormalization())

        model.add(LSTM(128, input_shape=((1, x)), return_sequences=True, activation="relu"))
        model.add(Dropout(0.2))
        model.add(BatchNormalization())

        model.add(LSTM(128, input_shape=((1, x)), activation="relu"))
        model.add(Dropout(0.2))
        model.add(BatchNormalization())

        model.add(Dense(32, activation="relu"))
        model.add(Dropout(0.2))

        model.add(Dense(2, activation="softmax"))
        optimizer = tf.keras.optimizers.Adam(lr = 0.001, decay = 1e-6)
        model.compile(loss="sparse_categorical_crossentropy",optimizer=optimizer)

        return model

    def train(self,x_train,y_train, batch_size, epochs):
        x_train = x_train.reshape(-1,1,len(x_train[0]))
        print("> Training model - ",self.model_name)
        self.model.fit(x_train,y_train,batch_size = batch_size, epochs = epochs)

    def evaluate(self,x_test,y_test):
        x_test = x_test.reshape(-1,1,len(x_test[0]))
        print("> Evaluating model - ",self.model_name)
        #Dự đoán lấy giá trị có khả xảy ra nhất
        predictions = np.array(tf.argmax(self.model.predict(x_test),1))

        '''
        expected_increase: số lần tăng
        expected_decrese: số lần giảm
        found_increase: số lần dự đoán tăng đúng
        found_decrese: số lần dự đoán giảm đúng
        '''
        expected_increase = 0
        found_increase = 0
        expected_decrese = 0
        found_decrese = 0

        for i in range(0,len(predictions)):
            if y_test[i] == 0:
                expected_decrese += 1
                if predictions[i] == y_test[i]:
                    found_decrese += 1
            else:
                expected_increase += 1
                if predictions[i] == y_test[i]:
                    found_increase += 1

        '''
        Accuracy: tỉ lệ dự đoán đúng
        Increase Acc: tỉ lệ dự đoán đúng thật sự tăng
        Decrese Acc: tỉ lệ dự đoán đúng thật sự giảm
        Loss: hàm mất mát
        '''
        accuracy = accuracy_score(y_test,predictions)
        print(">> Accuracy: ",accuracy)
        print(">> Increase Acc: ",(found_increase/expected_increase)," Decrese Acc: ",(found_decrese/expected_decrese))
        loss = self.model.evaluate(x_test,y_test)
        print("Loss re: ",loss)

    def predict(self,sample):
        sample = sample.reshape(-1, 1, len(sample[0]))
        prediction = np.array(tf.argmax(self.model.predict(sample),1))[0]

        return prediction

Tạo class Model có vai trò: <br>
+ Xây dựng mô hình
+ Train mô hình
+ Đánh giá mô hình 
+ Dự đoán 

In [5]:
class VirtualAccount:

    def __init__(self):
        self.usd_balance = 1000
        self.btc_amount = 0
        self.btc_balance = 0
        self.btc_price = 0
        self.bought_btc_at = 0
        self.last_transaction_was_sell = False

Tạo class VitualAccount để giả lập một tài khoản để trade

In [6]:
class AutoTrader:

    def __init__(self,model):
        self.advisor = model
        self.account = VirtualAccount()
        self.trade_amount = 100

    #Hàm mua btc
    def buy(self):
        prev_bought_at = self.account.bought_btc_at
        #gán giá mua dự đoán = giá mua tại trc đó
        if self.account.usd_balance - self.trade_amount >= 0:
        # Nếu số dư usd - giá trị gd >=0 (check đủ tiền gd ko)
            if prev_bougqht_at == 0 or self.account.last_transaction_was_sell or (prev_bought_at > self.account.btc_price):
            # Nếu giá mua dự đoán = 0 /hoặc/ giao dịch bán cuối cùng /hoặc/ giá mua dự đoán = giá mua trước đó (lơn hơn)> giá trị btc hiện tại (nghĩa là btc đang giảm, đặt lệnh mua)
                print(">> BUYING $",self.trade_amount," WORTH OF BITCOIN")
                # Mua "bao nhiu"
                self.account.btc_amount += self.trade_amount / self.account.btc_price
                # Số lượng btc += số lượng giao dịch/giá trị btc
                self.account.usd_balance -= self.trade_amount
                # Số dư usd -= số đã mua
                self.account.bought_btc_at = self.account.btc_price
                # Mua btc tại = giá btc lúc đó
                self.account.last_transaction_was_sell = False
                # giao dich cuối là bán = False
            else:
                print(">> Not worth buying more BTC at the moment")
        else:
            print(">> Not enough USD left in your account to buy BTC ")
    
    #Hàm bán btc
    def sell(self):
        if self.account.btc_balance - self.trade_amount >= 0:
        # Nếu số dư > số lượng gd (có btc để bán ko)
            if self.account.btc_price > self.account.bought_btc_at:
            # Nếu giá trị (bán) > giá mua tại (giá mua trc đó) -> bán (có lãi)   (giá btc hiện tại > giá mua trc đó ==> btc đang tăng, đặt lệnh bán)
                print(">> SELLING $",self.trade_amount," WORTH OF BITCOIN")
                self.account.btc_amount -= (self.trade_amount / self.account.btc_price)
                # Số lượng btc -= (số lượng giao dịch/ giá trị)
                self.account.usd_balance += self.trade_amount
                # Số dư usd += số lượng giao dịch (+ tiền vốn & lãi)
                self.account.last_transaction_was_sell = True
                # Giao dịch cuối là bán = True
            else:
                print(">> Declining sale: Not profitable to sell BTC")
        else:
            print(">> Not enough BTC left in your account to buy USD ")
    
    #Hàm trading & update số dư
    def runSimulation(self,samples,prices):
        print("> Trading Automatically for ",TESTING_MONTHS)
        day_count = 0
        for i in range(0,len(samples)):

            if i % 24 == 0: #Thống kê giao dịch trong 1 ngày
                day_count += 1
                print("#################################################################################################")
                print("#           Account Balance: $", (self.account.usd_balance + self.account.btc_balance), " BTC: $",
                      self.account.btc_balance, " USD: $", self.account.usd_balance, "")
                print("#################################################################################################")
                print("##########################################   DAY ",day_count,"   #########################################")


            if i % 6 == 0: #dự doán mỗi 6h
                # giá dự doán
                prediction = self.advisor.predict(np.array([samples[i]]))
                
                # giá btc thực tế
                btc_price = prices[i]

                if self.account.btc_price != 0:
                # Nếu giá thực tế khác 0
                    self.account.btc_balance = self.account.btc_amount * btc_price
                    # số dư btc = số lượng btc * giá trị btc

                self.account.btc_price = btc_price
                #cập nhật giá btc

                if prediction == 1:
                #nếu dự doán = 1 -> đặt lệnh mua
                    self.buy()
                else:
                #Còn lại, đặt lệnh bán
                    self.sell()

                self.account.btc_balance = self.account.btc_amount * btc_price

                #update lại số dư btc = số lượng * giá trị
                time.sleep(1)

        print("#################################################################################################")
        print("#           Account Balance: $", (self.account.usd_balance + self.account.btc_balance), " BTC: $",
              self.account.btc_balance, " USD: $", self.account.usd_balance, "")
        print("#################################################################################################")

Tạo class AutoTrader có vai trò: <br>
+ có chức năng mua
+ có chứ năng bán
+ hàm logic trade dựa theo dữ liệu mô hình đã dự đoán

### Đọc dữ liệu train test

In [7]:
dataset = Dataset()
print("> Creating Training Data for ", COIN_PAIR)
data = dataset.loadCoinData(COIN_PAIR, TRAINING_MONTHS)
x_train, y_train, _ = dataset.createTrainTestSets(COIN_PAIR, data, training_window=TRAINING_WINDOW, labeling_window=LABELING_WINDOW)

print("> Creating Testing Data for ", COIN_PAIR)
data = dataset.loadCoinData(COIN_PAIR, TESTING_MONTHS)
x_test, y_test, prices = dataset.createTrainTestSets(COIN_PAIR, data, training_window=TRAINING_WINDOW, labeling_window=LABELING_WINDOW)

> Dataset Class Initialized
> Creating Training Data for  BTC-USD
> Loading data for  BTC-USD
>> Loading month:  2018_06
>> Loading month:  2018_07
>> Loading month:  2018_08
>> Loading month:  2018_09
>> Loading month:  2018_10
>> Loading month:  2018_11
>> Loading month:  2018_12
>> Loading month:  2019_01
>> Loading month:  2019_02
>> Loading month:  2019_03
>> Loading month:  2019_04
>> Loading month:  2019_05
>> Loading month:  2019_06
>> Loading month:  2019_07
>> Loading month:  2019_08
>> Loading month:  2019_09
>> Loading month:  2019_10
>> Loading month:  2019_11
>> Loading month:  2019_12
>> Loading month:  2020_01
>> Loading month:  2020_02
>> Loading month:  2020_03
>> Loading month:  2020_04
>> Loading month:  2020_05
>> Loading month:  2020_06
>> Loading month:  2020_07
> Creating X,Y sets for  BTC-USD
> Finished Creating set - Size:  18969   18969  P:  9649  N:  9320
> Creating Testing Data for  BTC-USD
> Loading data for  BTC-USD
>> Loading month:  2020_08
>> Loading m

### Xây dựng mô hình và đánh giá

In [8]:
test_model = Model("AutoTraderAI", x_train)
test_model.train(x_train, y_train, batch_size=64, epochs=10)
test_model.evaluate(x_test,y_test)

> New model initialized:  AutoTraderAI
> Training model -  AutoTraderAI
Epoch 1/10
297/297 [==============================] - 8s 10ms/step - loss: 0.8458
Epoch 2/10
297/297 [==============================] - 3s 10ms/step - loss: 0.7087
Epoch 3/10
297/297 [==============================] - 3s 10ms/step - loss: 0.6960
Epoch 4/10
297/297 [==============================] - 3s 10ms/step - loss: 0.6930
Epoch 5/10
297/297 [==============================] - 3s 10ms/step - loss: 0.6923
Epoch 6/10
297/297 [==============================] - 3s 11ms/step - loss: 0.6897
Epoch 7/10
297/297 [==============================] - 3s 10ms/step - loss: 0.6896
Epoch 8/10
297/297 [==============================] - 3s 10ms/step - loss: 0.6882
Epoch 9/10
297/297 [==============================] - 3s 10ms/step - loss: 0.6863
Epoch 10/10
297/297 [==============================] - 3s 10ms/step - loss: 0.6869
> Evaluating model -  AutoTraderAI
>> Accuracy:  0.5538812785388127
>> Increase Acc:  0.6239669421487604  D

In [9]:
auto_trader = AutoTrader(test_model)
auto_trader.runSimulation(x_test, prices)

> Trading Automatically for  ['2020_08', '2020_09', '2020_10']
#################################################################################################
#           Account Balance: $ 1000  BTC: $ 0  USD: $ 1000 
#################################################################################################
##########################################   DAY  1    #########################################
>> Not enough BTC left in your account to buy USD 
>> BUYING $ 100  WORTH OF BITCOIN
>> BUYING $ 100  WORTH OF BITCOIN
>> Not worth buying more BTC at the moment
#################################################################################################
#           Account Balance: $ 984.7851292597604  BTC: $ 184.7851292597604  USD: $ 800 
#################################################################################################
##########################################   DAY  2    #########################################
>> SELLING $ 100  WORTH OF BITCOIN
>> Not

>> BUYING $ 100  WORTH OF BITCOIN
>> BUYING $ 100  WORTH OF BITCOIN
>> SELLING $ 100  WORTH OF BITCOIN
#################################################################################################
#           Account Balance: $ 1122.594247305923  BTC: $ 522.594247305923  USD: $ 600 
#################################################################################################
##########################################   DAY  17    #########################################
>> BUYING $ 100  WORTH OF BITCOIN
>> SELLING $ 100  WORTH OF BITCOIN
>> SELLING $ 100  WORTH OF BITCOIN
>> SELLING $ 100  WORTH OF BITCOIN
#################################################################################################
#           Account Balance: $ 1104.335354401116  BTC: $ 304.3353544011162  USD: $ 800 
#################################################################################################
##########################################   DAY  18    #####################################

>> BUYING $ 100  WORTH OF BITCOIN
>> Not worth buying more BTC at the moment
>> Not worth buying more BTC at the moment
#################################################################################################
#           Account Balance: $ 1245.3880536624486  BTC: $ 945.3880536624486  USD: $ 300 
#################################################################################################
##########################################   DAY  33    #########################################
>> Not worth buying more BTC at the moment
>> Not worth buying more BTC at the moment
>> Not worth buying more BTC at the moment
>> SELLING $ 100  WORTH OF BITCOIN
#################################################################################################
#           Account Balance: $ 1193.2068703496423  BTC: $ 793.2068703496423  USD: $ 400 
#################################################################################################
##########################################   DAY

>> SELLING $ 100  WORTH OF BITCOIN
>> BUYING $ 100  WORTH OF BITCOIN
>> BUYING $ 100  WORTH OF BITCOIN
#################################################################################################
#           Account Balance: $ 1320.798545020946  BTC: $ 1220.798545020946  USD: $ 100 
#################################################################################################
##########################################   DAY  49    #########################################
>> Not worth buying more BTC at the moment
>> SELLING $ 100  WORTH OF BITCOIN
>> BUYING $ 100  WORTH OF BITCOIN
>> BUYING $ 100  WORTH OF BITCOIN
#################################################################################################
#           Account Balance: $ 1271.8959945517133  BTC: $ 1271.8959945517133  USD: $ 0 
#################################################################################################
##########################################   DAY  50    #############################

>> SELLING $ 100  WORTH OF BITCOIN
>> BUYING $ 100  WORTH OF BITCOIN
>> Not worth buying more BTC at the moment
#################################################################################################
#           Account Balance: $ 1591.584044539448  BTC: $ 1491.584044539448  USD: $ 100 
#################################################################################################
##########################################   DAY  64    #########################################
>> SELLING $ 100  WORTH OF BITCOIN
>> BUYING $ 100  WORTH OF BITCOIN
>> Declining sale: Not profitable to sell BTC
>> BUYING $ 100  WORTH OF BITCOIN
#################################################################################################
#           Account Balance: $ 1459.0641605143692  BTC: $ 1459.0641605143692  USD: $ 0 
#################################################################################################
##########################################   DAY  65    #################

>> Not worth buying more BTC at the moment
>> BUYING $ 100  WORTH OF BITCOIN
#################################################################################################
#           Account Balance: $ 1565.6179571905554  BTC: $ 1065.6179571905554  USD: $ 500 
#################################################################################################
##########################################   DAY  79    #########################################
>> BUYING $ 100  WORTH OF BITCOIN
>> SELLING $ 100  WORTH OF BITCOIN
>> BUYING $ 100  WORTH OF BITCOIN
>> BUYING $ 100  WORTH OF BITCOIN
#################################################################################################
#           Account Balance: $ 1697.8418977542592  BTC: $ 1397.8418977542592  USD: $ 300 
#################################################################################################
##########################################   DAY  80    #########################################
>> BUYING $ 100  W